How to run:


create your enviornment: 
1. conda create --name modeling python=3.9

activate your enviornment:

2. conda activate modeling

install the required packages: 

3. conda install -c conda-forge sentence-transformers
4. conda install numpy matplotlib

In [93]:
data = {}

# download data
path = "/Users/xkabot/Desktop/workspace/Math_Modeling_Notes/final_project/synthetic_data_1000.csv"
with open(path) as f:

    # read the first line 
    f.readline()
    count = 0
    for line in f:
        
        sentence,max_distance,hobby_relatability_score,job_relatability_score = line.split(",")
        data[count] = [sentence,max_distance,int(hobby_relatability_score),int(job_relatability_score)]
        count += 1

In [94]:
import numpy as np

# set seed of PRNG to stop our distances arr from changing
np.random.seed(42)


# create and display our distance array
O_distances = np.random.uniform(0, len(data), (len(data), len(data)))
symmetric_array = (O_distances + O_distances.T) / 2
np.fill_diagonal(O_distances, 1)
print(O_distances)

# this is to access every single sentence within our dictionary 
sentences = []
max_distances = []
for i in data:
    sentences.append(data[i][0])
    max_distances.append(data[i][1])
    print(data[i][0])
    print(data[i][1])




[[  1.         950.71430641 731.99394181 ... 136.81863092 950.23735382
  446.00577296]
 [185.13292884   1.         872.94583588 ...  68.95801636  57.05472115
  282.18707469]
 [261.70568374 246.97879907   1.         ... 309.78785921 290.04553196
  871.41403419]
 ...
 [ 85.26213476  92.03751325 694.38212955 ...   1.         913.30033476
  175.38920929]
 [760.21961064 963.82646192   4.77477808 ... 334.70456942   1.
  956.14838931]
 [114.29114855 159.80353711 825.72709072 ... 418.07198203 428.67125669
    1.        ]]
A writer who enjoys gardening.
25
The nurse who enjoys photography.
25
A teacher who enjoys cooking.
50
A teacher who enjoys painting.
50
The writer who enjoys writing.
100
A programmer who enjoys hiking.
50
A teacher who enjoys gardening.
25
The teacher who enjoys cycling.
25
A teacher who enjoys photography.
25
A designer who enjoys cycling.
50
The artist who enjoys hiking.
50
The teacher who enjoys cycling.
100
A writer who enjoys painting.
25
The doctor who enjoys cycling

In [95]:
# import our sentence transformer
from sentence_transformers import SentenceTransformer

In [96]:
# we can now convert ot vector embeddings using SBERT 
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(sentences)

# calculate cosine similarity 
similarities = model.similarity(embeddings, embeddings)

# print the similarities 
print(similarities)


tensor([[1.0000, 0.3605, 0.4893,  ..., 0.4043, 0.6305, 0.4868],
        [0.3605, 1.0000, 0.3674,  ..., 0.2523, 0.3752, 0.3217],
        [0.4893, 0.3674, 1.0000,  ..., 0.2434, 0.3080, 0.4538],
        ...,
        [0.4043, 0.2523, 0.2434,  ..., 1.0000, 0.6900, 0.6680],
        [0.6305, 0.3752, 0.3080,  ..., 0.6900, 1.0000, 0.7364],
        [0.4868, 0.3217, 0.4538,  ..., 0.6680, 0.7364, 1.0000]])


In [97]:
type(O_distances[0][0])
type(max_distances[0])

str

In [98]:
# now let's eliminate people from the response if they are not near the other person  
for i in range(len(similarities)):
    for j in range(len(similarities[0])):
        if not ((int(O_distances[i][j]) <= int(max_distances[i])) and (int(O_distances[j][i]) <= int(max_distances[j]))):
            similarities[i][j] = 0
similarities = similarities - np.eye(len(similarities))
print(similarities)

tensor([[ 3.5763e-07,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -1.1921e-07,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -2.3842e-07,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.5763e-07,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
         -1.1921e-07,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], dtype=torch.float64)


In [99]:
def top_n(matrix, n=3):
    matrix = np.array(matrix)
    if matrix.ndim != 2:
        raise ValueError("Input matrix must be a 2D array.")
    top = {}
    for i, row in enumerate(matrix):
        exclude = np.ones_like(row, dtype=bool)  
        exclude[i] = False 
        exclude[row == 0] = False  
        
        idx = np.where(exclude)[0]
        vals = row[exclude]
        
        if len(vals) > 0:
            srt_idx = idx[np.argsort(-vals)][:n]  
            top[i] = srt_idx.tolist()
        else:
            top[i] = []  
    return top

top_3 = top_n(similarities, n=3)


In [100]:
print(top_3)

{0: [470, 128, 840], 1: [901, 264], 2: [297, 692], 3: [66, 912, 874], 4: [721, 411, 900], 5: [985, 525, 646], 6: [754], 7: [434, 732, 582], 8: [760, 680], 9: [191, 663, 835], 10: [718, 661, 529], 11: [413, 645, 981], 12: [657], 13: [398, 25, 34], 14: [399, 194, 228], 15: [99, 184], 16: [985], 17: [308, 670, 636], 18: [186, 727, 354], 19: [801, 278, 477], 20: [84, 885], 21: [], 22: [263, 432, 595], 23: [], 24: [183, 378, 827], 25: [580, 431, 652], 26: [552, 883], 27: [240, 698, 314], 28: [974, 839, 580], 29: [857, 366, 410], 30: [513], 31: [191, 166], 32: [137, 958, 528], 33: [179, 737], 34: [783, 429, 571], 35: [], 36: [], 37: [805, 291], 38: [290, 727], 39: [680, 775, 477], 40: [205, 787, 427], 41: [578, 46], 42: [104, 187], 43: [], 44: [831, 781, 185], 45: [754, 911, 373], 46: [451, 41], 47: [967, 464], 48: [905, 894], 49: [875, 735], 50: [730, 603, 436], 51: [772], 52: [290, 299, 239], 53: [449, 431, 183], 54: [13], 55: [465, 280, 429], 56: [433, 75], 57: [814, 191], 58: [497], 59: 

In [101]:
# we create a scoring system to test the reliablity of this using our synthetic data 
# for each vector in our dict 
#   if the matches have at least one relatoion
#       our score is increased by .33 
#
possible = len(data)
score = 0
total = 0
for count, matches in top_3.items():
    # Extract the current vector's max_distance
    max_distance = data[count][1]
    
    # Check if any match satisfies the hobby or job relatability condition
    valid_matches = [
        match for match in matches
        if data[match][2] >= 0.5 or data[match][3] >= 0.5  # Relatability thresholds
    ]


    # Increment the score by 0.33 if there are valid matches
    
    if len(valid_matches) != 0:
        print(valid_matches)
        score += 1

    for i in valid_matches:
        total += 1


# this scores means that at least 904 people got at least a single valid match where they were 
# recommended someones profile that they had at least one person in common a job/hobby 
print("number of people with at least one match:", score)
print("number of people: ", len(data))

# this should be out of 1000 samples so 90% of people got at least one valid match. 

print("number of total valid matches: ", total)
# this total means that of the 904 people to get a match 2,165 total valid matches were created using the algo
# of the possible 3000 this is a matching rate of over 60%.




[470, 128, 840]
[901, 264]
[297, 692]
[66, 912, 874]
[721, 411, 900]
[985, 525, 646]
[754]
[434, 732, 582]
[760, 680]
[191, 663, 835]
[718, 661, 529]
[413, 645, 981]
[657]
[398, 25, 34]
[399, 194, 228]
[99, 184]
[985]
[308, 670, 636]
[186, 727, 354]
[801, 278, 477]
[84, 885]
[263, 432, 595]
[183, 378, 827]
[580, 431, 652]
[552, 883]
[240, 698, 314]
[974, 839, 580]
[857, 366, 410]
[513]
[191, 166]
[137, 958, 528]
[179, 737]
[783, 429, 571]
[805, 291]
[290, 727]
[680, 775, 477]
[205, 787, 427]
[578, 46]
[104, 187]
[831, 781, 185]
[754, 911, 373]
[451, 41]
[967, 464]
[905, 894]
[875, 735]
[730, 603, 436]
[772]
[290, 299, 239]
[449, 431, 183]
[13]
[465, 280, 429]
[433, 75]
[814, 191]
[497]
[907, 100]
[462, 508, 139]
[327]
[282, 164, 490]
[161, 809]
[274, 571]
[754, 227, 749]
[3, 617, 483]
[82, 378, 657]
[821, 879, 755]
[162, 427]
[464, 86, 728]
[482, 797, 760]
[947, 270]
[914, 56, 39]
[68]
[201, 911, 749]
[476]
[629, 464, 190]
[662, 915, 451]
[67]
[378, 248, 995]
[683, 387, 372]
[72, 704]
